In [ ]:
# Import a subset of Amazon office product review dataset. For our example, we’ll be working with a subset of the Amazon Review Dataset
# that consists of around 50,000 reviews of items from Amazon’s office supplies category (http://jmcauley.ucsd.edu/data/amazon/links.html)
import pandas as pd
import random

file_path = "/mnt/sda/haystack/amazon_data/reviews_Office_Products_5.json"

reviews = pd.read_json(file_path, lines = True)
# print(reviews.columns)

# Extract review text and ids and convert them into a dictionary. The meta field contains item_id
# which will be used for filtering later.
texts = reviews.reviewText.values
ids = reviews.asin.values

dicts = [{'content': text, 'meta': {'item_id': id_}} for text, id_ in zip(texts, ids)]

random.choice(dicts)

In [ ]:
# Since we’re going to work with the dense DPR retrieval method, we let the preprocessor split
# our reviews into chunks of length 100 and an overlap of five words.
from haystack.nodes import PreProcessor

processor = PreProcessor(
    split_by = 'word', 
    split_length = 100,
    split_respect_sentence_boundary = False,
    split_overlap = 5)
flattened_docs = processor.process(dicts)

random.choice(flattened_docs)

In [ ]:
# Load the data in elastic
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host = "localhost", port = 9200, username = "elastic", 
    password = "Dkmh=pOI=CSukfWwOoxh", index = "document", scheme = "https", verify_certs = True,
    ca_certs = "/home/tanvir/work/qa-experiment/http_ca.crt")
document_store.delete_documents()
document_store.write_documents(flattened_docs)
print(f"Loaded {document_store.get_document_count()} documents")

In [ ]:
# Initiate the dense passage retriever and update embeddings.
from haystack.nodes import DensePassageRetriever
retriever = DensePassageRetriever(
    document_store = document_store,
    query_embedding_model = "facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model = "facebook/dpr-ctx_encoder-single-nq-base",
    max_seq_len_query = 64,
    max_seq_len_passage = 256,
    batch_size = 16,
    use_gpu = True,
    embed_title = True,
    use_fast_tokenizers = True,
)
document_store.update_embeddings(retriever)

In [ ]:
from haystack.nodes import FARMReader
import os

# Supress the warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

reader = FARMReader(model_name_or_path = "deepset/roberta-base-squad2", use_gpu = True, return_no_answer = True)

In [ ]:
# Set up the pipeline
from haystack.pipelines import ExtractiveQAPipeline
pipeline = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# Quick peek at the item with most reviews
reviews.groupby('asin').size().sort_values(ascending = False).head(1)

In [ ]:
# In total we have 311 reviews for item id 'B0010T3QT2' (some kind of envelope).
# Lets invoke the pipeline with the filter with item id.
from haystack.utils import print_answers
import time
filter = {'item_id': ['B0010T3QT2']}
q = 'How well does this envelope stick?'
start_time = time.process_time()
answers = pipeline.run(q, params = {"Retriever": {"top_k": 30}, "Reader": {"top_k": 10}, "filters": filter})
elapsed_time = time.process_time() - start_time
print(f"Time taken with filter: {elapsed_time} seconds")
print_answers(answers, details = "all")

In [ ]:
# Now lets run the same query without filters. It should take slightly longer but the answers won't be super relevant
start_time = time.process_time()
answers = pipeline.run(q, params = {"Retriever": {"top_k": 30}, "Reader": {"top_k": 10}})
elapsed_time = time.process_time() - start_time
print(f"Time taken without filter: {elapsed_time} seconds")
print_answers(answers, details = "all")

In [ ]:
# Its possible to filter across multiple products to find answers about a product category
filter_mutiple = {'item_id': ['B00006IBLJ', 'B000GHJM9C', 'B000CS787S']}
q2 = 'Can things still break when they\'re wrapped in bubble wrap?'
start_time = time.process_time()
answers = pipeline.run(q2, params = {"Retriever": {"top_k": 100}, "Reader": {"top_k": 3}, "filters": filter_mutiple})
elapsed_time = time.process_time() - start_time
print(f"Time taken with multi-filter: {elapsed_time} seconds")
print_answers(answers, details = "all")

In [ ]:
# QA is great but Haystack also works as a just retriever pipeline. Lets run the same set of experiments
# for the retriever pipeline.

In [ ]:
# Filtering on one product id
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

search_pipeline = DocumentSearchPipeline(retriever)
search_query = "Easy to seal but hard to use"
start_time = time.process_time()
search_result = search_pipeline.run(search_query, params = {"Retriever": {"top_k": 10}, "filters": filter})
elapsed_time = time.process_time() - start_time
print(f"Time taken with multi: {elapsed_time} seconds")
print_documents(search_result, max_text_len = 100, print_name = True, print_meta = True)

In [ ]:
# Retriever without filter
start_time = time.process_time()
search_result = search_pipeline.run(search_query, params = {"Retriever": {"top_k": 10}})
elapsed_time = time.process_time() - start_time
print(f"Time taken without filter: {elapsed_time} seconds")
print_documents(search_result, max_text_len = 100, print_name = True, print_meta = True)

In [ ]:
# Its possible to filter across multiple products to find answers about a product category
filter_mutiple = {'item_id': ['B00006IBLJ', 'B000GHJM9C', 'B000CS787S']}
start_time = time.process_time()
search_result = search_pipeline.run(search_query, params = {"Retriever": {"top_k": 10}, "filters": filter_mutiple})
elapsed_time = time.process_time() - start_time
print(f"Time taken with multi-filter: {elapsed_time} seconds")
print_documents(search_result, max_text_len = 100, print_name = True, print_meta = True)

In [120]:
# Lets plot how adding more filters is related to query execution time.
def calculate_retrieval_time(item_id_list, query_list):
    total_time = 0.0
    for q in query_list:
        start_time = time.process_time()
        search_result = search_pipeline.run(q, params = {"Retriever": {"top_k": 10}, "filters": {'item_id': item_id_list}})
        total_time += (time.process_time() - start_time)
    return total_time

query_list = [
    'Office supplies are expensive',
    'Stapler is broken',
    'Printer does not work',
    'Erasers are too abrasive'
    'Easy to seal but hard to use'
]
item_id_list = ['B00006IBLJ', 'B000GHJM9C', 'B000CS787S',
                'B00004Z59A', 'B000KKMO90', 'B004JDI1I2',
                'B003155XYO', 'B002VUCCNU', 'B00066FHNI', 
                'B003VNE25M']

query_item_id_list = []
for i in range(11):
    total = calculate_retrieval_time(query_item_id_list, query_list)
    print(f"Number of filters: {len(query_item_id_list)} - Execution time: {total}")
    if i < 10:
        query_item_id_list.append(item_id_list[i])


Number of filters: 0 - Execution time: 0.12494182799969167
Number of filters: 1 - Execution time: 0.10182105299986688
Number of filters: 2 - Execution time: 0.09505063100004918
Number of filters: 3 - Execution time: 0.09579995000012786
Number of filters: 4 - Execution time: 0.09451272500041341
Number of filters: 5 - Execution time: 0.09400194499994541
Number of filters: 6 - Execution time: 0.09281263899993064
Number of filters: 7 - Execution time: 0.09322232299996358
Number of filters: 8 - Execution time: 0.09555610299980799
Number of filters: 9 - Execution time: 0.09571709099986947
Number of filters: 10 - Execution time: 0.09547573099962392
